In [24]:
!pip install plotly

In [139]:
import numpy as np
import pandas as pd

# 필요한 Surprise 알고리즘 불러오기
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBasic, KNNBaseline, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.dataset import DatasetAutoFolds
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import PredictionImpossible

# DATA LOAD

In [5]:
#ratings=pd.read_csv("rating_complete.csv")

In [6]:
#ratings.to_csv("ratings_processed.csv", index=False, header=False)

In [7]:
#ratings=pd.read_csv("ratings_processed.csv")

In [74]:
anime=pd.read_csv("anime_full.csv")

In [75]:
ratings=pd.read_csv("ratings_new2.csv")

In [76]:
column_name = ['user_id', 'anime_id', 'rating']

data = ratings.values.tolist()

new_ratings = pd.DataFrame(data, columns=column_name)

In [41]:
new_ratings

,user_id,anime_id,rating
0,478,3613,7
1,478,5462,6
2,478,33023,7
3,478,176,5
4,478,5332,6
...,...,...,...
4354363,352930,671,7
4354364,352930,1017,8
4354365,352930,10465,6
4354366,352930,10845,6


In [4]:
ratings.to_csv("ratings_new3.csv", index=False, header=False)

In [5]:
#from sklearn.model_selection import train_test_split

#train, test = train_test_split(ratings, test_size=0.9)

In [6]:
#train.to_csv("ratings_train.csv")

In [7]:
reader = Reader(line_format= 'user item rating', sep=',', rating_scale=(0, 10))
data = Dataset.load_from_file('ratings_new3.csv', reader=reader)

# DATA  PREPROCESSING

In [21]:
ratings.groupby('user_id')['rating'].count().reset_index().sort_values('rating',ascending=False)

,user_id,rating
165818,189037,15455
142622,162615,14864
59682,68042,13462
249092,283786,12778
227994,259790,9996
...,...,...
176586,201316,1
176574,201300,1
85501,97555,1
176568,201293,1


In [22]:
ratings.groupby('anime_id')['rating'].count().reset_index().sort_values('rating',ascending=False)

,anime_id,rating
1390,1535,182375
7437,16498,169794
6603,11757,161192
4629,6547,141127
10420,30276,138924
...,...,...
12799,35153,1
16559,42465,1
16643,42757,1
15509,39627,1


In [16]:
ratings.columns=["user_id", "anime_id", "rating"]

In [17]:
ratings

,user_id,anime_id,rating
0,0,570,7
1,0,2762,9
2,0,431,8
3,0,578,10
4,0,433,6
...,...,...,...
55651546,353404,502,8
55651547,353404,987,4
55651548,353404,225,8
55651549,353404,243,7


In [41]:
min_ratings = 1000

filter_animes = ratings_new['anime_id'].value_counts() >= min_ratings
filter_users = ratings_new['user_id'].value_counts() >= min_ratings

filter_animes = filter_animes[filter_animes].index.tolist()
filter_users = filter_users[filter_users].index.tolist()

ratings_new2 = ratings_new[ratings_new['anime_id'].isin(filter_animes)]
ratings_new2 = ratings_new[ratings_new['user_id'].isin(filter_users)]

print('The original data frame shape:\t{}'.format(ratings_new.shape))
print('The new data frame shape:\t{}'.format(ratings_new2.shape))

The original data frame shape:	(54207031, 3)
The new data frame shape:	(4354369, 3)


In [42]:
ratings_new2.groupby('user_id')['rating'].count().reset_index().sort_values('rating',ascending=False)

,user_id,rating
1683,189037,4635
2581,283786,4579
625,68042,4573
2524,277841,4450
1437,162615,4406
...,...,...
676,73750,1000
1416,160098,1000
1932,214717,1000
3160,350407,1000


In [43]:
ratings_new2.groupby('anime_id')['rating'].count().reset_index().sort_values('rating',ascending=False)

,anime_id,rating
1816,6547,3031
2397,11757,3001
2736,16498,2982
2925,19815,2944
1556,4224,2927
...,...,...
1328,2817,103
4639,39761,96
680,951,91
874,1316,81


In [27]:
print('(기존 애니메이션 편수: ',len(set(ratings['anime_id'])),')', ' - ', '(필터링 하고 애니메이션 편수: ',len(set(ratings_new2['anime_id'])),')', ' = ',len(set(ratings['anime_id'])) - len(set(ratings_new2['anime_id'])))

(기존 애니메이션 편수:  16872 )  -  (필터링 하고 애니메이션 편수:  16424 )  =  448


In [28]:
print('(기존 사용자 수: ',len(set(ratings['user_id'])),')', ' - ', '(필터링 하고 난 사용자 수: ',len(set(ratings_new2['user_id'])),')', ' = ',len(set(ratings['user_id'])) - len(set(ratings_new2['user_id'])))

(기존 사용자 수:  310059 )  -  (필터링 하고 난 사용자 수:  293842 )  =  16217


In [44]:
len(ratings_new2)

4354369

In [45]:
ratings_new2.reset_index(drop=True, inplace=True)
ratings_new2 = ratings_new2.loc[:, ["user_id", "anime_id", "rating"]]
ratings_new2

,user_id,anime_id,rating
0,478,4224,9
1,478,3613,7
2,478,5462,6
3,478,33023,7
4,478,176,5
...,...,...,...
4354364,352930,671,7
4354365,352930,1017,8
4354366,352930,10465,6
4354367,352930,10845,6


In [4]:
range(len(ratings))

range(0, 4354368)

In [21]:
ratings_new = ratings[ratings['anime_id'].isin(list(anime["MAL_ID"][:]))]

In [28]:
ratings_new

,user_id,anime_id,rating
0,0,570,7
1,0,2762,9
2,0,431,8
3,0,578,10
4,0,433,6
...,...,...,...
55651546,353404,502,8
55651547,353404,987,4
55651548,353404,225,8
55651549,353404,243,7


In [140]:
#ratings_new2.to_csv("ratings_new2.csv", index=False, header=False)
reader = Reader(line_format= 'user item rating', sep=',', rating_scale=(0, 10))
data = DatasetAutoFolds('ratings_new2.csv', reader=reader)

# DATA VIZ

In [17]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go

# py파일로 실행할 경우 노트북에서 실행시키기 위한 함수.
# init_notebook_mode(connected=True)




# 데이터프레임에서 한 컬럼만 뽑아낸 시리즈를 종류별로 카운트하고, 인덱스에 따라 내림차순 정렬
data = ratings['rating'].value_counts().sort_index(ascending=False)


# x에는 index로 평점 1~5를 나타내고, y는 평점 당 카운트 값이다.
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / ratings.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
               
               
# 레이아웃 생성하기.
layout = dict(title = 'Distribution Of {} anime-ratings'.format(ratings.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# 그래프 생성하기
fig = go.Figure(data=[trace], layout=layout)
plot(fig) 

'temp-plot.html'

In [18]:
# 영화 한 편당 평점 개수. groupby 함수로 카운팅한다.
data = ratings.groupby('anime_id')['rating'].count().clip(upper=100)

# trace 생성하기.
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))
# 레이아웃 생성하기
layout = go.Layout(title = 'Distribution Of Number of Ratings Per anime (Clipped at 100)',
                   xaxis = dict(title = 'Number of Ratings Per anime'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# 그래프 생성하기.
fig = go.Figure(data=[trace], layout=layout)
plot(fig)

'temp-plot.html'

In [19]:
ratings.groupby('anime_id')['rating'].count().reset_index().sort_values('rating', ascending=False)[:10]

,anime_id,rating
1390,1535,182375
7437,16498,169794
6603,11757,161192
4629,6547,141127
10420,30276,138924
1428,1575,137291
3559,4224,135524
3965,5114,134197
8135,19815,129009
8631,22319,128822


In [20]:
# 사용자별 평점 개수
data = ratings.groupby('user_id')['rating'].count().clip(upper=50)

# trace 생성하기.
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))
# 레이아웃 생성하기.
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User (Clipped at 50)',
                   xaxis = dict(title = 'Ratings Per User'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
plot(fig)

'temp-plot.html'

# Cross Validate

In [47]:
algo = SVD(random_state=42)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0569  1.0558  1.0586  1.0562  1.0546  1.0564  0.0013  
MAE (testset)     0.7804  0.7795  0.7816  0.7804  0.7787  0.7801  0.0010  
Fit time          228.92  224.35  230.14  230.45  228.88  228.55  2.19    
Test time         15.94   14.01   14.95   13.89   16.59   15.07   1.06    


{'test_rmse': array([1.05689137, 1.05583642, 1.05859039, 1.05623985, 1.05458802]),
 'test_mae': array([0.78043228, 0.77945525, 0.78158814, 0.78043399, 0.77870529]),
 'fit_time': (228.9232542514801,
  224.35309720039368,
  230.144606590271,
  230.45479035377502,
  228.8847782611847),
 'test_time': (15.935636043548584,
  14.008955478668213,
  14.953330516815186,
  13.887745380401611,
  16.58536386489868)}

In [59]:
algo = SlopeOne()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2050  1.2064  1.2083  1.2053  1.2071  1.2064  0.0012  
MAE (testset)     0.9011  0.9022  0.9024  0.9021  0.9032  0.9022  0.0007  
Fit time          245.80  250.10  253.03  252.26  288.77  257.99  15.59   
Test time         1094.92 1059.25 1086.89 1086.61 1075.80 1080.69 12.33   


{'test_rmse': array([1.20502044, 1.20641868, 1.2082842 , 1.20534435, 1.20711455]),
 'test_mae': array([0.90114576, 0.9021617 , 0.90238792, 0.90213942, 0.9032269 ]),
 'fit_time': (245.80135941505432,
  250.10209465026855,
  253.03271746635437,
  252.2572376728058,
  288.7696590423584),
 'test_time': (1094.9235260486603,
  1059.2464685440063,
  1086.8932321071625,
  1086.605796098709,
  1075.7992794513702)}

In [51]:
algo = NMF(random_state=42)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.8497  1.8517  1.8523  1.8552  1.8527  1.8523  0.0018  
MAE (testset)     1.5833  1.5855  1.5861  1.5887  1.5860  1.5859  0.0017  
Fit time          250.07  267.74  252.49  236.70  246.12  250.62  10.11   
Test time         17.81   17.54   16.19   17.10   17.98   17.32   0.64    


{'test_rmse': array([1.84968475, 1.85171673, 1.85227326, 1.85523629, 1.85268956]),
 'test_mae': array([1.58330144, 1.58549484, 1.58611255, 1.5887227 , 1.58597829]),
 'fit_time': (250.06591725349426,
  267.73959827423096,
  252.48865699768066,
  236.69538164138794,
  246.12001729011536),
 'test_time': (17.8092098236084,
  17.540966987609863,
  16.185835123062134,
  17.099680185317993,
  17.983340740203857)}

In [52]:
algo = KNNBaseline(random_state=42)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1567  1.1581  1.1605  1.1592  1.1567  1.1582  0.0015  
MAE (testset)     0.8616  0.8626  0.8638  0.8633  0.8615  0.8625  0.0009  
Fit time          263.38  255.76  256.62  254.17  255.98  257.18  3.20    
Test time         937.02  940.94  946.35  960.83  968.17  950.66  11.91   


{'test_rmse': array([1.15668149, 1.15811931, 1.16049818, 1.15918187, 1.15667966]),
 'test_mae': array([0.86159806, 0.86256142, 0.8637587 , 0.86329855, 0.86153138]),
 'fit_time': (263.380229473114,
  255.7564160823822,
  256.62239956855774,
  254.17269134521484,
  255.98461890220642),
 'test_time': (937.0167663097382,
  940.9448580741882,
  946.3521747589111,
  960.8255679607391,
  968.1655452251434)}

In [53]:
algo = KNNBasic(random_state=42)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1937  1.1921  1.1922  1.1915  1.1905  1.1920  0.0010  
MAE (testset)     0.8847  0.8846  0.8842  0.8837  0.8827  0.8840  0.0007  
Fit time          234.98  226.10  229.95  229.31  231.58  230.38  2.91    
Test time         925.52  961.51  905.33  956.98  951.23  940.11  21.39   


{'test_rmse': array([1.19369661, 1.19211751, 1.19224852, 1.19150355, 1.19050682]),
 'test_mae': array([0.88466441, 0.88458233, 0.88424209, 0.88371881, 0.88268771]),
 'fit_time': (234.98097944259644,
  226.10450506210327,
  229.95067954063416,
  229.31077075004578,
  231.57623863220215),
 'test_time': (925.5240576267242,
  961.5128438472748,
  905.3283383846283,
  956.9763872623444,
  951.2298991680145)}

In [54]:
algo = KNNWithMeans(random_state=42)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1675  1.1687  1.1692  1.1677  1.1695  1.1685  0.0008  
MAE (testset)     0.8721  0.8731  0.8729  0.8728  0.8737  0.8729  0.0005  
Fit time          224.33  230.44  234.28  233.82  235.60  231.70  4.06    
Test time         954.32  961.29  949.79  954.34  972.43  958.43  7.90    


{'test_rmse': array([1.16754187, 1.16866707, 1.16923419, 1.16768625, 1.16949101]),
 'test_mae': array([0.87211615, 0.87305807, 0.87293758, 0.87279863, 0.87374246]),
 'fit_time': (224.33285307884216,
  230.44231271743774,
  234.2839584350586,
  233.8182075023651,
  235.6035234928131),
 'test_time': (954.3168609142303,
  961.2940361499786,
  949.7904493808746,
  954.3426477909088,
  972.426198720932)}

In [55]:
algo = KNNWithZScore(random_state=42)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithZScore on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1766  1.1752  1.1734  1.1786  1.1784  1.1764  0.0020  
MAE (testset)     0.8709  0.8700  0.8690  0.8721  0.8719  0.8708  0.0012  
Fit time          243.03  231.49  237.92  236.89  242.37  238.34  4.18    
Test time         990.19  944.49  972.10  1006.24 1046.20 991.85  34.05   


{'test_rmse': array([1.17659324, 1.17517264, 1.17343271, 1.17858996, 1.17839248]),
 'test_mae': array([0.87091641, 0.87001342, 0.86902274, 0.87209179, 0.87188368]),
 'fit_time': (243.0261836051941,
  231.49236178398132,
  237.9162585735321,
  236.89057803153992,
  242.37281250953674),
 'test_time': (990.1937158107758,
  944.4892494678497,
  972.1029720306396,
  1006.2416141033173,
  1046.2013189792633)}

In [57]:
algo = BaselineOnly()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2084  1.2095  1.2068  1.2083  1.2096  1.2085  0.0010  
MAE (testset)     0.9040  0.9039  0.9028  0.9035  0.9050  0.9038  0.0007  
Fit time          21.20   23.93   23.11   23.48   24.09   23.16   1.04    
Test time         11.30   14.65   10.23   12.03   13.41   12.33   1.55    


{'test_rmse': array([1.20844149, 1.2095443 , 1.2067855 , 1.2082706 , 1.20958985]),
 'test_mae': array([0.90401602, 0.90388889, 0.90276093, 0.9034973 , 0.90500029]),
 'fit_time': (21.19957971572876,
  23.93129873275757,
  23.11083459854126,
  23.475759029388428,
  24.09245276451111),
 'test_time': (11.303348779678345,
  14.64543342590332,
  10.233033180236816,
  12.032972812652588,
  13.411656618118286)}

In [60]:
algo = CoClustering(random_state=42)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1929  1.1910  1.1914  1.1925  1.1941  1.1924  0.0011  
MAE (testset)     0.8920  0.8904  0.8917  0.8918  0.8926  0.8917  0.0007  
Fit time          89.13   94.85   95.43   88.84   90.10   91.67   2.87    
Test time         13.22   20.20   13.46   18.28   15.44   16.12   2.73    


{'test_rmse': array([1.19285547, 1.19102556, 1.1914443 , 1.19248381, 1.19407417]),
 'test_mae': array([0.89197704, 0.89036753, 0.89172446, 0.89181729, 0.89257064]),
 'fit_time': (89.1253387928009,
  94.84870386123657,
  95.43240809440613,
  88.84267663955688,
  90.10373973846436),
 'test_time': (13.21584153175354,
  20.19637107849121,
  13.457183361053467,
  18.281070947647095,
  15.43964171409607)}

# Grid Search
* **algo_class**: 알고리즘 이름
* **param_grid**: 파라미터 조합을 딕셔너리 형태로 표시
* **measures**: 비교할 측정지표 (rmse, mae)
* **cv**: cross-validation sample 개수
* **refit**: True이면 best parameter를 전체 데이터에 적용한 측정지표(measures의 첫번째 지표)를 계산
* **return_train_measures**: True이면 train set에 대한 측정지표를 계산
* **n_jobs**: parallel training을 진행할 maximum job의 수(-1: 모든 CPU 사용, 1: parallel이 없음.)
* **pre_dispatch**: 메모리 full을 막기 위해 미리 dispatch할 parallel job의 수를 조정(default = 2 * n_jobs)
* **joblob_verbose**: message를 표시하는 정도

## KNN 기반 알고리즘(아이템 기반 최근점 이웃 CF)의 파라미터
* **k**: 이웃의 크기
* **name**: 사용할 유사도의 종류를 나타내는 문자열. 디폴트는 'MSD'.
    * 평균제곱차이 유사도 (Mean Squared Difference Similarity)
    * 코사인 유사도 (Cosine Similarity)
    * 피어슨 유사도 (Pearson Similarity)
    * 피어슨-베이스라인 유사도 (Pearson-Baseline Similarity)
* **user_based**: True면 사용자 기반, False면 상품 기반.

## SVD 기반 알고리즘(행렬 분해를 이용한 잠재요인 CF)의 파라미터
* **n_epochs**: SGD의 반복 횟수 지정
* **n_factors**: SVD의 잠재 요인 K의 크기 지정
* **lr_all**: 학습률
* **reg_all**: 규제 텀 계수

In [ ]:
###### 최적화할 파라미터들을 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [17, 18, 19], 'n_factors': [3000, 4500, 6000] }
 
# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=-1)
gs.fit(data)
 
# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

# RecSys 구현

In [141]:
trainset = data.build_full_trainset() # 전체 데이터를 학습데이터로 생성함.

In [147]:
algo = SVD(n_epochs=17, n_factors=1500, random_state=42)
algo.fit(trainset)

In [98]:
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [132]:
predictions

[Prediction(uid='478', iid='10122', r_ui=7.040737475395401, est=5.9245652225332135, details={'was_impossible': False}),
 Prediction(uid='478', iid='851', r_ui=7.040737475395401, est=7.1524514428089265, details={'was_impossible': False}),
 Prediction(uid='478', iid='15005', r_ui=7.040737475395401, est=5.983420051080612, details={'was_impossible': False}),
 Prediction(uid='478', iid='30745', r_ui=7.040737475395401, est=7.138247537545802, details={'was_impossible': False}),
 Prediction(uid='478', iid='21241', r_ui=7.040737475395401, est=5.971710200899473, details={'was_impossible': False}),
 Prediction(uid='478', iid='23385', r_ui=7.040737475395401, est=7.150221378677125, details={'was_impossible': False}),
 Prediction(uid='478', iid='10893', r_ui=7.040737475395401, est=5.696565759865514, details={'was_impossible': False}),
 Prediction(uid='478', iid='15359', r_ui=7.040737475395401, est=6.1018761516878115, details={'was_impossible': False}),
 Prediction(uid='478', iid='6172', r_ui=7.04073

In [145]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('478', '10122', 5.9245652225332135),
 ('478', '851', 7.1524514428089265),
 ('478', '15005', 5.983420051080612)]

In [ ]:
# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인.
animeids = new_ratings[new_ratings['user_id']==478]['anime_id']
if animeids[animeids==42].count() == 0:
    print('사용자 아이디 9는 애니메이션 아이디 42의 평점 없음')
 
print(anime[anime['MAL_ID']==42])

In [152]:
# 개별 사용자의 아이템에 대한 추천 평점 예측
uid = str(478)
iid = str(42)
 
pred = algo.predict(uid, iid, verbose=True)
pred                                        
# uid: 사용자 아이디, iid: 아이템 아이디, r_ui: 실제 평점, est: 예측 평점               

user: 478        item: 42         r_ui = None   est = 6.86   {'was_impossible': False}


Prediction(uid='478', iid='42', r_ui=None, est=6.8573073726850975, details={'was_impossible': False})

In [87]:
def get_unseen_animelist(user_id):
    
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 애니메이션을 리스트로 생성
    seen_animes = new_ratings[new_ratings['user_id']== user_id]['anime_id'].tolist()
     
    # 모든 애니메이션들의 MAL_ID를 리스트로 생성.
    total_animes = anime['MAL_ID'].tolist()
     
    # 모든 애니메이션들의 MAL_ID 중 이미 평점을 매긴 애니메이션의 MAL_ID를 제외하여 리스트로 생성
    unseen_animes= [anime for anime in total_animes if anime not in seen_animes]
    print('평점 매긴 애니메이션 수:',len(seen_animes), '추천 대상 애니메이션 수:',len(unseen_animes),'전체 애니메이션 수:',len(total_animes))
     
    return unseen_animes
 
unseen_animes = get_unseen_animelist(478)

평점 매긴 애니메이션 수: 1204 추천 대상 애니메이션 수: 10977 전체 애니메이션 수: 12181


In [151]:
def recomm_anime_by_surprise(algo, user_id, unseen_animes, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 애니메이션에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(user_id), str(anime_id)) for anime_id in unseen_animes]
     
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
     
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
     
    # top_n으로 추출된 애니메이션의 정보 추출. 애니메이션 아이디, 추천 예상 평점, 제목 추출
    top_anime_ids = [int(pred.iid) for pred in top_predictions]
    top_anime_rating = [pred.est for pred in top_predictions]
    top_anime_names = anime[anime.MAL_ID.isin(top_anime_ids)]['Name']
    top_anime_preds = [ (id, title, rating) for id, title, rating in zip(top_anime_ids, top_anime_names, top_anime_rating)]
     
    return top_anime_preds

unseen_animes = get_unseen_animelist(15000)
top_anime_preds = recomm_anime_by_surprise(algo, 15000, unseen_animes, top_n=10)
print('##### Top-10 추천 애니메이션 리스트 #####')
 
for top_anime in top_anime_preds:
    print(top_anime[1], ":", top_anime[2])

평점 매긴 애니메이션 수: 0 추천 대상 애니메이션 수: 12181 전체 애니메이션 수: 12181
##### Top-10 추천 애니메이션 리스트 #####
Ginga Eiyuu Densetsu : 8.79672550350547
Gintama : 8.655544101932412
Code Geass: Hangyaku no Lelouch R2 : 8.640951737191434
Clannad: After Story : 8.588815669465333
Fullmetal Alchemist: Brotherhood : 8.546451458768663
Steins;Gate : 8.533221651349523
Gintama' : 8.495415123636144
Hunter x Hunter (2011) : 8.493630944043892
Koe no Katachi : 8.481103922413599
Gintama° : 8.447609797729699
